In [1]:
from keras import applications
from evec_scan.tools import evectools as evt
import pandas as pd
import os
import numpy as np
from keras import applications

SyntaxError: '{' was never closed (evectools.py, line 232)

In [2]:
df = pd.DataFrame()
instance_dir = 'evec_scan/instances/test1'
vid_file = 'vid3.mp4'
instance_id = 'test1'
group_size = 2
vidpath = 'evec_scan/instances/test1/vid3.mp4'
description_models = ['moondream'] #['moondream', 'llava-llama3']
description_embedding_models = ['mxbai-embed-large'] #['mxbai-embed-large','nomic-embed-text', 'all-minilm']
#######################################################################################################


In [3]:
video_instance = evt.FrameExtractor(vidpath=vidpath)
frame_group = video_instance.get_random_frame_group(size=group_size)
frame_group_instance = video_instance.FrameGroup(frame_extractor=video_instance, group_set=frame_group)
frames_output_path = os.path.join(instance_dir, 'frames')
frame_path_list = frame_group_instance.save_group(output_dir=frames_output_path, group_name=instance_id)
df['frame_path'] = frame_path_list
df.index = list(range(len(frame_path_list)))
########################################################################################################


files were succesfully saved


In [4]:
for desc_model in description_models: 
    descriptions_per_model = []
    description_columns = []
    for frame_path in df['frame_path']:
        descriptor = evt.ImgDescriptor(model=desc_model,frame_path=frame_path)
        description = descriptor.get_description()
        descriptions_per_model.append(description)
    df[f'{desc_model}_desc'] = descriptions_per_model
    description_columns.append(f'{desc_model}_desc')
#######################################################################################################3


In [5]:
df

,frame_path,moondream_desc
0,evec_scan/instances/test1\frames\test1_6522.jpg,"\nThe image depicts a city street at night, wi..."
1,evec_scan/instances/test1\frames\test1_31816.jpg,"\nThe image depicts a city street at night, wi..."


In [6]:
for dir in description_columns:
    description_evec_output_dir = os.path.join(instance_dir, 'desc_evecs', dir)
    os.makedirs(description_evec_output_dir)
    for text_evec_model in description_embedding_models:
        description_evec_arr_per_model_per_desc_model = np.empty(group_size, dtype=object)
        i = 0
        for description in df[dir]:
            text_evec_scanner = evt.TextEvecScanner(model=text_evec_model, text=description)
            text_evec = text_evec_scanner.get_evec()
            description_evec_arr_per_model_per_desc_model[i]=text_evec
            i += 1
        output_path = os.path.join(description_evec_output_dir,f'{text_evec_model}.npy')
        np.save(file=output_path, arr=description_evec_arr_per_model_per_desc_model)






In [8]:
image_embedding_models = [applications.VGG16, applications.DenseNet121]
for image_evec_models in image_embedding_models:
    image_evec_arr_per_model_per_img_model = np.empty(group_size, dtype=object)
    img_evec_scanner = evt.ImgEVecScanner()
    img_evec_scanner.add_model(Keras_applications_model=image_evec_models)
    for frame_path in df['frame_path']:
        i = 0
        res_dict = img_evec_scanner.get_models_evecs(frame_path=frame_path, lin_method='GAP')
        for name, evec in zip(res_dict['model_name'],res_dict['embedding_vector']):
            output_path_img_evec = os.path.join(instance_dir,f'{name}.npy')
            image_evec_arr_per_model_per_img_model[i] = evec[0]
            i += 1
    np.save(file=output_path_img_evec, arr=image_evec_arr_per_model_per_img_model)

the new models list is [<Functional name=vgg16, built=True>]
models that will be used have indexes: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 391ms/step
the new models list is [<Functional name=densenet121, built=True>]
models that will be used have indexes: [0]
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step


In [10]:
df_output_path = os.path.join(instance_dir, 'instance_data.csv')
df.to_csv(df_output_path, index=True)

In [2]:
d = {"a":["um", "dois"], "b":["um2", "dois2"]}
for x, y in zip(d["a"],d["b"]):
    print((x,y))

('um', 'um2')
('dois', 'dois2')


In [1]:
from evec_scan.tools import evectools as evt

In [ ]:
m = evt.ImgEVecScanner()
res = m.get_clip_evec( model="ViT-B/32",frame_path="evec_scan/instances/test1/frames/test1_848.jpg")
print(res)

TypeError: ImgEVecScanner.get_clip_evec() missing 1 required positional argument: 'frame_path'